In [5]:
from ebm import *

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix


%matplotlib widget

# Adult dataset

In [6]:
# Load Adult dataset from UCI
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

df = pd.read_csv(url, names=columns, na_values=' ?', skipinitialspace=True)
df.dropna(inplace=True)

# Preprocess data
df['sex'] = LabelEncoder().fit_transform(df['sex'])  # Male=1, Female=0
df['income'] = (df['income'] == '>50K').astype(int)  # Binary classification

# Select key features
features = ['age', 'education-num', 'hours-per-week', 'sex', 'capital-gain']
X = df[features]
y = df['income']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create gender masks using sample weights
male_weights = (X_train['sex'] == 1).astype(float)
female_weights = (X_train['sex'] == 0).astype(float)

print(np.sum(male_weights))
print(np.sum(female_weights))

# Train gender-specific EBM models
male_model = EBMClassifier(n_cycles=1000, learning_rate=0.1, n_bins=256, max_depth=3, binning_strategy="uniform", smoothing_window=11)
male_model.fit(X_train, y_train, sample_weight=male_weights)
male_model.set_feature_names(features)

female_model = EBMClassifier(n_cycles=1000, learning_rate=0.1, n_bins=256, max_depth=3, binning_strategy="uniform", smoothing_window=11)
female_model.fit(X_train, y_train, sample_weight=female_weights)
female_model.set_feature_names(features)

normal_model = EBMClassifier(n_cycles=10, learning_rate=0.1, n_bins=256, max_depth=3, binning_strategy="uniform", smoothing_window=11)
normal_model.fit(X_train, y_train)
normal_model.set_feature_names(features)

# Evaluate performance
def evaluate_model(model, X, y, name):
    preds = model.predict(X)
    acc = accuracy_score(y, preds)
    print(f"\n{name} Model:")
    print(f"Overall Accuracy: {acc:.2f}")
    print("Gender Performance:")
    print(f"Male Accuracy: {accuracy_score(y[X['sex']==1], preds[X['sex']==1]):.2f}")
    print(f"Female Accuracy: {accuracy_score(y[X['sex']==0], preds[X['sex']==0]):.2f}")

print("=== Test Set Performance ===")
evaluate_model(male_model, X_test, y_test, "Male-Trained")
evaluate_model(female_model, X_test, y_test, "Female-Trained")
evaluate_model(normal_model, X_test, y_test, "Normal")

# Analyze sex feature contributions
print("\nSex Feature Contributions:")
print("Male Model:", male_model.feature_graphs[features.index('sex')][1])
print("Female Model:", female_model.feature_graphs[features.index('sex')][1])

# Visualize feature impacts
vis = EBMVisualizer([male_model, female_model, normal_model], model_names=["Male-Trained", "Female-Trained", "Normal"])
vis.show()

17403.0
8645.0


EBM round-robin cycles:   0%|          | 0/1000 [00:00<?, ?it/s]

EBM round-robin cycles:   0%|          | 0/1000 [00:00<?, ?it/s]

EBM round-robin cycles:   0%|          | 0/10 [00:00<?, ?it/s]

=== Test Set Performance ===

Male-Trained Model:
Overall Accuracy: 0.81
Gender Performance:
Male Accuracy: 0.77
Female Accuracy: 0.90

Female-Trained Model:
Overall Accuracy: 0.80
Gender Performance:
Male Accuracy: 0.75
Female Accuracy: 0.90

Normal Model:
Overall Accuracy: 0.76
Gender Performance:
Male Accuracy: 0.70
Female Accuracy: 0.89

Sex Feature Contributions:
Male Model: [-0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766
 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766
 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766
 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766
 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766
 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766
 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766
 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766 -0.08698766
 -0.08698766 -0.08698766 -0.08698

# German Dataset

In [3]:
# Load German Credit Dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
columns = [
    'checking_status', 'duration', 'credit_history', 'purpose', 'credit_amount',
    'savings_account', 'employment', 'installment_rate', 'personal_status_sex',
    'other_debtors', 'present_residence', 'property', 'age', 'other_installment_plans',
    'housing', 'existing_credits', 'job', 'num_maintenance', 'telephone', 'foreign_worker', 'target'
]

df = pd.read_csv(url, sep=' ', names=columns, header=None)

# Preprocessing
# Create binary sex feature (Male=1, Female=0)
df['sex'] = df['personal_status_sex'].apply(lambda x: 1 if x in ['A91', 'A93', 'A94'] else 0)

# Convert target to binary (Good credit=1, Bad credit=0)
df['target'] = df['target'].replace({1: 1, 2: 0})

# Select and encode features
features = ['age', 'sex', 'credit_amount', 'duration', 'checking_status', 'savings_account']
categorical_features = ['checking_status', 'savings_account']

# Label encode categorical features
le = LabelEncoder()
for col in categorical_features:
    df[col] = le.fit_transform(df[col])

X = df[features]
y = df['target']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create gender-specific sample weights
male_weights = (X_train['sex'] == 1).astype(float)
female_weights = (X_train['sex'] == 0).astype(float)

# Train models
male_model = EBMClassifier(n_cycles=1000, learning_rate=0.1, n_bins=256, max_depth=3, smoothing_window=11, binning_strategy="uniform")
male_model.fit(X_train, y_train, sample_weight=male_weights)
male_model.set_feature_names(features)

female_model = EBMClassifier(n_cycles=1000, learning_rate=0.1, n_bins=256, max_depth=3, smoothing_window=11, binning_strategy="uniform")
female_model.fit(X_train, y_train, sample_weight=female_weights)
female_model.set_feature_names(features)

normal_model = EBMClassifier(n_cycles=1000, learning_rate=0.1, n_bins=256, max_depth=3, smoothing_window=11, binning_strategy="uniform")
normal_model.fit(X_train, y_train)
normal_model.set_feature_names(features)

print("=== Test Set Performance ===")
evaluate_model(male_model, X_test, y_test, "Male")
evaluate_model(female_model, X_test, y_test, "Female")
evaluate_model(normal_model, X_test, y_test, "Normal")

# Analyze feature contributions
print("Sex Feature Contributions:")
print(f"Male Model: {male_model.feature_graphs[features.index('sex')][1]}")
print(f"Female Model: {female_model.feature_graphs[features.index('sex')][1]}")

# Visualize differences
vis = EBMVisualizer([male_model, female_model, normal_model], model_names=["Male-Trained", "Female-Trained", "Normal"])
vis.show()

EBM round-robin cycles:   0%|          | 0/1000 [00:00<?, ?it/s]

EBM round-robin cycles:   0%|          | 0/1000 [00:00<?, ?it/s]

EBM round-robin cycles:   0%|          | 0/1000 [00:00<?, ?it/s]

=== Test Set Performance ===

Male Model:
Overall Accuracy: 0.71
Gender Performance:
Male Accuracy: 0.72
Female Accuracy: 0.70

Female Model:
Overall Accuracy: 0.69
Gender Performance:
Male Accuracy: 0.66
Female Accuracy: 0.77

Normal Model:
Overall Accuracy: 0.72
Gender Performance:
Male Accuracy: 0.72
Female Accuracy: 0.73
Sex Feature Contributions:
Male Model: [0.08944209 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209
 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209
 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209
 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209
 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209
 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209
 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209
 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209
 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209 0.08944209
 0.08944209 0.08944209 0.089442

# Taiwan Dataset

In [7]:
data = pd.read_csv("data/UCI_Credit_Card.csv")
data = data.drop(columns=["ID"])
data = data.rename(columns={"default.payment.next.month": "default"})
display(data)

X = data.drop(columns=["default"])
y = data["default"]

# Label encode categorical features:
le = LabelEncoder()
for col in X.select_dtypes(include="object").columns:
    X[col] = le.fit_transform(X[col])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create gender-specific sample weights
male_weights = (X_train['SEX'] == 1).astype(float)
female_weights = (X_train['SEX'] == 2).astype(float)

# Train models
male_model = EBMClassifier(n_cycles=100, learning_rate=0.1, n_bins=256, max_depth=3, smoothing_window=11, binning_strategy="uniform")
male_model.fit(X_train, y_train, sample_weight=male_weights)
male_model.set_feature_names(X.columns)

female_model = EBMClassifier(n_cycles=100, learning_rate=0.1, n_bins=256, max_depth=3, smoothing_window=11, binning_strategy="uniform")
female_model.fit(X_train, y_train, sample_weight=female_weights)
female_model.set_feature_names(X.columns)

normal_model = EBMClassifier(n_cycles=100, learning_rate=0.1, n_bins=256, max_depth=3, smoothing_window=11, binning_strategy="uniform")
normal_model.fit(X_train, y_train)
normal_model.set_feature_names(X.columns)

print("=== Test Set Performance ===")

# Visualize feature impacts
vis = EBMVisualizer([male_model, female_model, normal_model], model_names=["male", "female", "normal"])
vis.show()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000.0,1,3,1,39,0,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,150000.0,1,3,2,43,-1,-1,-1,-1,0,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,30000.0,1,2,2,37,4,3,2,-1,0,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,80000.0,1,3,1,41,1,-1,0,0,0,...,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


EBM round-robin cycles:   0%|          | 0/100 [00:00<?, ?it/s]

EBM round-robin cycles:   0%|          | 0/100 [00:00<?, ?it/s]

EBM round-robin cycles:   0%|          | 0/100 [00:00<?, ?it/s]

=== Test Set Performance ===


# Cancer dataset

In [8]:
data = pd.read_csv("data/Cancer_Data.csv")
data.drop(columns=["Unnamed: 32", "id"], inplace=True)
data["diagnosis"] = data["diagnosis"].map({"M": 1, "B": 0})

X = data.drop(columns="diagnosis")
y = data["diagnosis"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model = EBMClassifier(n_cycles=1000, max_depth=3, learning_rate=1, smoothing_window=5)
model.set_feature_names(X.columns)
model.fit(X_train, y_train)

preds = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))

# Visualize feature impacts
vis = EBMVisualizer([model], model_names=["Cancer Model"])
vis.show()

EBM round-robin cycles:   0%|          | 0/1000 [00:00<?, ?it/s]

Accuracy: 0.9649122807017544


# Synthetic dataset:

In [10]:
np.random.seed(42)

n_samples = 10000

# Feature 1: Linear relationship
num1 = np.random.normal(0, 1, n_samples)

# Feature 2: Quadratic relationship
num2 = np.random.uniform(-2, 5, n_samples)

# Feature 3: Categorical relationship
cat1 = np.random.choice(['A', 'B', 'C'], size=n_samples, p=[0.3, 0.4, 0.3])

# Generate ground truth log-odds
log_odds = (
    2.0 * num1 +  # Linear effect
    (-3.0 * num2 + 1.5 * np.square(num2)) +  # Quadratic effect
    np.select([cat1 == 'A', cat1 == 'B'], [2.0, 0.0], default=-2.0)  # Categorical effect
)

# Convert to probabilities
probs = 1 / (1 + np.exp(-log_odds))

# Generate binary labels
y = np.random.binomial(1, probs)

# Create DataFrame
df = pd.DataFrame({
    'num1': num1,
    'num2': num2,
    'cat1': cat1,
    'target': y
})

In [11]:
X = df.drop(columns='target')
X['cat1'] = LabelEncoder().fit_transform(df['cat1'])
display(X.head())
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = EBMClassifier(n_bins=256, max_depth=3, n_cycles=100, smoothing_window=11, binning_strategy="uniform")
model.fit(X_train, y_train)
model.set_feature_names(X.columns)

preds = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))

# Visualize feature impacts
vis = EBMVisualizer([model], model_names=["Simulated Model"])
vis.show()

,num1,num2,cat1
0,0.496714,0.248786,0
1,-0.138264,-1.278866,1
2,0.647689,-1.497347,2
3,1.523030,-1.340068,2
4,-0.234153,2.080086,0


EBM round-robin cycles:   0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 0.903
